In [ ]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd, listdir

In [ ]:
listdir()

['.config', 'cats-and-dogs.zip', 'sample_data']

In [14]:
path_cats_and_dogs = f"{getcwd()}/cats-and-dogs.zip"
print(path_cats_and_dogs)
#shutil.rmtree('/tmp')
os.mkdir("/tmp")



/content/cats-and-dogs.zip


In [13]:
zip_ref = zipfile.ZipFile(path_cats_and_dogs, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

BadZipFile: ignored

In [ ]:
cat_path = "/tmp/PetImages/Cat"
dog_path = "/tmp/PetImages/Dog"

print(len(listdir(cat_path)))
print(len(listdir(dog_path)))

FileNotFoundError: ignored

In [ ]:
training_path = "/tmp/cats-v-dogs/training"
testing_path = "/tmp/cats-v-dogs/testing"

try:
  os.mkdir('/tmp/cats-v-dogs')
  os.mkdir('/tmp/cats-v-dogs/training')
  os.mkdir('/tmp/cats-v-dogs/testing')
  os.mkdir('/tmp/cats-v-dogs/training/cats')
  os.mkdir('/tmp/cats-v-dogs/training/dogs')
  os.mkdir('/tmp/cats-v-dogs/testing/dogs')
  os.mkdir('/tmp/cats-v-dogs/testing/cats')
except OSError:
  print(OSError)

In [ ]:
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  
  # Get all non_zero images
  non_zero_images = []
  for path in os.listdir(SOURCE):
    if (os.path.getsize(SOURCE + path) > 0):
      non_zero_images.append(path)
  
  print(f"There are {len(non_zero_images)} non-zero images of {SOURCE.split('/')[-1]}")

  # Find how many X images the SPLIT_SIZE demands
  testing_amount = len(non_zero_images) - (len(non_zero_images) * SPLIT_SIZE)
  print(f"Testing amount: {testing_amount}")

  # Sample X amount of images into TESTING and the rest into TRAINING
  testing_images = random.sample(non_zero_images, int(testing_amount))
  training_images = non_zero_images
  
  # Insert the images AND prep the training_images
  for path in testing_images:
    training_images.remove(path)
    shutil.copy(SOURCE + path, TESTING)

  for path in training_images:
    shutil.copy(SOURCE + path, TRAINING)
  
  print(f"Length of {TRAINING}: {len(os.listdir(TRAINING))} \nLength of {TESTING}: {len(os.listdir(TESTING))}")

In [ ]:
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Define the model

First: Evaluate some characteristics of the image-files using **matplotlib**

In [ ]:
from matplotlib import image

random_image = random.choice(os.listdir(CAT_SOURCE_DIR))
random_image_path = CAT_SOURCE_DIR + random_image

image = image.imread(random_image_path)
print(image.shape)


In [ ]:
# Make sure the first Conv2D layer have the same input_size as the ImageGenerators

model = tf.keras.Sequential([
                             # Conv + Maxpooling
                             tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2, 2)),

                             # Flatten
                             tf.keras.layers.Flatten(),

                             # Dense layers
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
TRAINING_DIR = training_path
train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(200, 200),
    batch_size=10,
    class_mode='binary'
)

VALIDATION_DIR = testing_path
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(200, 200),
    batch_size=10,
    class_mode='binary'
)


In [ ]:
history = model.fit(
    train_generator,
    epochs=2,
    verbose=2,
    validation_data=validation_generator
)